# 문서 전처리 프로세스

문서 청킹과 문맥 검색을 포함한 전처리 파이프라인을 구현하는 과정을 안내해드리도록 하겠습니다.

In [ ]:
import os
from dotenv import load_dotenv
from libs.rag_utils import Context_Processor, OpenSearch_Manager
import yaml

load_dotenv()

## OpenSearch Manager 초기화하기

In [ ]:
os_manager = OpenSearch_Manager()
print(f"사용 가능한 인덱스: {os_manager.index_list}")

## 처리 파라미터 설정하기

In [ ]:
# 모델 설정 불러오기
with open("libs/config.yml", "r") as file:
    model_config = yaml.safe_load(file)

# 처리 파라미터 설정
use_context_retrieval = True
index_name = "contextual_test" if use_context_retrieval else "test"
chunk_size = 500
overlap = 50
max_document_len = 50000  # context_retrieval 사용시에만 적용

# 모델 설정
embed_model = list(model_config['embedding_models'].keys())[0]
embed_model_id = model_config['embedding_models'][embed_model]['model_id']
context_model = list(model_config['models'].keys())[1]  # 두 번째 모델을 문맥용으로 사용
context_model_id = model_config['models'][context_model]['model_id']
bedrock_region = 'us-west-2'

## Context Processor 초기화하기

In [ ]:
context_processor = Context_Processor(
    os_manager=os_manager,
    embed_model=embed_model_id,
    bedrock_region=bedrock_region,
    index_name=index_name,
    chunk_size=chunk_size,
    overlap=overlap,
    use_context_retrieval=use_context_retrieval,
    context_model=context_model_id if use_context_retrieval else None,
    max_document_len=max_document_len if use_context_retrieval else None
)

## PDF 문서 처리하기

In [ ]:
# PDF 파일 경로를 지정해주세요
pdf_path = "sample.pdf"

# 파일 열기 및 처리
with open(pdf_path, 'rb') as file:
    context_processor.process_file(
        file=file,
        index_action="기존 인덱스 덮어쓰기",
        start_page=1,
        end_page=None  # 모든 페이지 처리
    )

## 처리 결과 확인하기

In [ ]:
# 생성된 인덱스 확인
os_manager.refresh_index_list()
print(f"업데이트된 인덱스: {os_manager.index_list}")

# 문맥 검색 사용시 토큰 사용량 확인
if use_context_retrieval:
    import json
    with open(f"{index_name}_token_usage.json", 'r') as f:
        token_usage = json.load(f)
        print("\n토큰 사용량:")
        print(json.dumps(token_usage['total_usage'], indent=2))